# COGS 188 - Project Proposal

# Project Description

You have the choice of doing either (1) an AI solve a problem style project or (2) run a Special Topics class on a topic of your choice.  If you want to do (2) you should fill out the _other_ proposal for that. This is the proposal description for (1).

You will design and execute a machine learning project. There are a few constraints on the nature of the allowed project. 
- The problem addressed will not be a "toy problem" or "common training students problem" like 8-Queens or a small Traveling Salesman Problem or similar
- If its the kind of problem (e.g., RL) that interacts with a simulator or live task, then the problem will have a reasonably complex action space. For instance, a wupus world kind of thing with a 9x9 grid is definitely too small.  A simulated mountain car with a less complex 2-d road and simplified dynamics seems like a fairly low achievement level.  A more complex 3-d mountain car simulation with large extent and realistic dynamics, sure sounds great!
- If its the kind of problem that uses a dataset, then the dataset will have >1k observations and >5 variables. I'd prefer more like >10k observations and >10 variables. A general rule is that if you have >100x more observations than variables, your solution will likely generalize a lot better. The goal of training an unsupervised machine learning model is to learn the underlying pattern in a dataset in order to generalize well to unseen data, so choosing a large dataset is very important.
- The project must include some elements we talked about in the course
- The project will include a model selection and/or feature selection component where you will be looking for the best setup to maximize the performance of your AI system. Generally RL tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible. 
- You will evaluate the performance of your AI system using more than one appropriate metric
- You will be writing a report describing and discussing these accomplishments


Feel free to delete this description section when you hand in your proposal.

# Names

Hopefully your team is at least this good. Obviously you should replace these with your names.

- Ashesh Kaji
- Abhay Anand
- Varun Pillai
- Ansh Bhatnagar

# Abstract 
The goal of this project is to train a Reinforcement Learning (RL) Classifier on autonomous vehicles. We plan to use DonkeyCar Simulator to navigate the "Warren Feild" track and collect our data. The data we plan to collect includes visual input from the front-facing camera, speed and steering angle change. These will be collected during training in the simulated environment. We implement and compare two deep RL algorithms: Actor-Critic and Proximal Policy Optimization (PPO), both designed for continuous action spaces. Moreover we plan to include a simple Q learning model to show how inefficient/ ineffective it is for more complex problems. We will use the gathered data to train agents to take optimal actions such as steering left/right, acceleration, and braking based on the cars current position. Performance will be evaluated using key metrics such as cumulative reward, lap completion time, and DonkeySim's own accuracy rating. By comparing these metrics across different models and training scenarios, we aim to determine which RL method provides the most robust and efficient control for autonomous driving in simulated environments.

# Original Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Background

Autonomous driving has emerged as a rapidly evolving field, as recent advances in computing power
and machine learning continue to push the boundaries of vehicle autonomy<a name="koutnik"></a>[<sup>[1]</sup>](#koutniknote).
A major breakthrough was the introduction of deep reinforcement learning methods capable of learning
complex control policies from high-dimensional data, such as pixel inputs<a name="mnih"></a>[<sup>[2]</sup>](#mnihnote).

Among various simulation tools, DonkeyCar Simulator ("DonkeySim") provides a relatively accessible
environment where researchers can collect training data in a less resource-intensive, hobby-focused
setting<a name="donkeycar"></a>[<sup>[3]</sup>](#donkeycarnote). DonkeySim offers a front-facing camera stream,
speed readings, and steering angle data—sufficient for exploring end-to-end RL pipelines.

Concurrent work in policy optimization techniques, such as Proximal Policy Optimization (PPO),
has improved training stability for continuous control tasks, making them more suitable
for real-world driving problems<a name="schulman"></a>[<sup>[4]</sup>](#schulmannotenote). By leveraging
vision-based RL, robust network architectures, and user-friendly simulators like DonkeySim,
researchers aim to accelerate the development of autonomous vehicle control systems.

Why is this important? Autonomous driving stands to improve road safety, increase mobility,
and reduce congestion. However, it also introduces unique challenges in perception, planning,
and control. Studying reinforcement learning in this domain is crucial for advancing algorithms
that can handle high-dimensional state spaces and continuous action controls, ultimately bringing
us closer to reliable self-driving cars.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data
## Original Instructions
You should have a strong idea of what dataset(s) will be used to accomplish this project. 

If you know what (some) of the data you will use, please give the following information for each dataset:
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc will be needed

If you don't yet know what your dataset(s) will be, you should describe what you desire in terms of the above bullets.

## Data
We will be using the CARLA open-source gym environment for our project. The CARLA simulator is a platform for autonomous driving research, providing a realistic urban driving environment with various weather conditions, traffic scenarios, and sensor modalities.


## Proposed Solution
We propose to implement a deep reinforcement learning (RL) approach using DonkeySim as the simulation environment. 

Key components include:
1. **Baseline Model (Monte Carlo Q-Learning)**: A simple Q-learning algorithm using lookup tables as a baseline.
2. **Actor-Critic Model**: A deep RL approach with separate networks for the actor (action selection) and the critic (value estimation).
3. **Proximal Policy Optimization (PPO)**: A robust deep RL algorithm designed to handle continuous control tasks.

Each model will be evaluated based on cumulative reward, lap completion time, collision count, tracking error, and a convergence metric. 

The simulation environment, DonkeySim, will provide the necessary sensor data and metrics to measure performance.

# Proposed Code (Explanatory)

Below is a high-level pseudocode outlining our intended implementation for each approach. Note that this is only explanatory; no full implementation is provided.

```python
# Pseudocode for PPO using DonkeySim
initialize DonkeySim environment
for each training episode:
    reset environment
    while episode not terminated:
        capture state from front camera, speed, steering angle
        compute policy (actor network) to decide action
        execute action in DonkeySim
        observe next state and reward
        store (state, action, reward) tuple
        if episode ends, update policy using PPO loss

# Pseudocode for Baseline Q-Learning
initialize lookup table for Q-values
for each training episode:
    reset environment
    while episode not terminated:
        capture state and discretize it
        select action using epsilon-greedy policy
        execute action and get reward from DonkeySim
        update Q-table based on reward

# Pseudocode for Actor-Critic Model
initialize actor and critic networks
for each training episode:
    reset environment
    while episode not terminated:
        get state from DonkeySim
        actor selects action; critic evaluates state
        execute action and receive reward
        store experience
        periodically update actor and critic based on advantage

# Note: The above pseudocode serves to explain our approach and does not constitute a full working implementation.
```


# Original Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Evaluation Metrics

For this study, we will focus on several key metrics that quantify performance and robustness
of autonomous driving RL models. The following metrics are commonly referenced in prior work
and can be applied to both simpler baselines (e.g., Monte Carlo Q-learning) and more advanced
methods (Actor-Critic, PPO, etc.) within DonkeyCar Simulator:

1. Cumulative Reward
   ------------------
   Definition:
     The total sum of rewards obtained over an episode or across training. A higher cumulative
     reward typically indicates more efficient navigation and adherence to safety constraints
     (e.g., staying on track, avoiding collisions)<a name="mnih"></a>[<sup>[2]</sup>](#mnihnote).

   Mathematical Representation:
     R_total = Σ (r_t)  for  t = 0 to T,
     where r_t is the reward at time step t and T is the total number of steps in the episode.

2. Lap Completion Time
   -------------------
   Definition:
     The time taken by the agent to complete a single lap (or multiple laps) in the DonkeySim
     environment. Lower times indicate faster and often more optimal driving behavior.
     DonkeyCar simulator logs track completion times automatically<a name="donkeycar"></a>[<sup>[3]</sup>](#donkeycarnote).

   Mathematical/Logical Form:
     - Time is measured in seconds from the lap start to completion.
     - This metric is a real-valued measure of efficiency (↓ is better).

3. Collision Count
   ---------------
   Definition:
     The number of collisions or off-track events per episode. This directly relates to safety
     and the agent’s ability to avoid obstacles. DonkeySim enables detection of collisions
     (or track deviations) through its built-in sensors.<a name="donkeycarAgain"></a>[<sup>[3]</sup>](#donkeycarnote)

   Quantifiable Measure:
     collision_count = Σ (binary_collision_indicator_t)
       for t = 0 to T
       (1 if a collision occurs at time t, otherwise 0)

4. Tracking Error
   --------------
   Definition:
     Measures how closely the agent adheres to the desired lane or trajectory. This metric
     is particularly useful in DonkeySim, which can log lateral deviation from a predefined
     track boundary<a name="koutnik"></a>[<sup>[1]</sup>](#koutniknote).

   Mathematical Representation (example for lateral deviation):
     tracking_error = (1 / T) * Σ (|lane_center - agent_position_t|)
       for t = 1 to T

5. Policy Convergence Rate
   ------------------------
   Definition:
     Reflects how quickly each model converges to a stable policy. Methods like Proximal Policy
     Optimization (PPO) often track the KL divergence between successive policy updates to gauge
     stability<a name="schulman"></a>[<sup>[4]</sup>](#schulmannotenote).

   Potential Approach:
     - Observe episode rewards over training; define convergence when ∆Reward < ε for N consecutive episodes.
     - Alternatively, use KL divergence thresholds between policy iterations.

By comparing these quantifiable metrics across algorithms and experimental conditions,
we can gauge which approaches best balance safety, efficiency, and reliable control in DonkeySim.

# Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination. Get creative!

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
* Regular updates via WhatsApp, weekly check-ins, quick status updates when problems/issues arise 
* Set realistic deadlines, and communicate on time if a problem comes up
* Decision-making should be done collaboratively, discussions for major decisions
* Address conflicts respectfully

# Project Timeline Proposal

Replace this with something meaningful that is appropriate for your needs. It doesn't have to be something that fits this format.  It doesn't have to be set in stone... "no battle plan survives contact with the enemy". But you need a battle plan nonetheless, and you need to keep it updated so you understand what you are trying to accomplish, who's responsible for what, and what the expected due dates are for each item.

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic (Pelé) | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets (Beckenbaur)  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data ,do some EDA (Maradonna) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin programming for project (Cruyff) | Discuss/edit project code; Complete project |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Carlos)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="koutniknote"></a>1.[^](#koutnik): Koutník, J., Schmidhuber, J., & Gómez, F. (2014).
Evolving deep unsupervised convolutional networks for vision-based reinforcement learning.
[arXiv preprint](https://arxiv.org/abs/1312.6120)<br>

<a name="mnihnote"></a>2.[^](#mnih): Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., et al. (2015).
Human-level control through deep reinforcement learning. *Nature*, 518(7540), 529–533.
[Link](https://www.nature.com/articles/nature14236)<br>

<a name="donkeycarnote"></a>3.[^](#donkeycar): DonkeyCar Documentation.
[Link](https://docs.donkeycar.com/)<br>

<a name="schulmannotenote"></a>4.[^](#schulman): Schulman, J., Wolski, F., Dhariwal, P., Radford, A., & Klimov, O. (2017).
Proximal Policy Optimization Algorithms.
[arXiv preprint](https://arxiv.org/abs/1707.06347)<br>
